# Stock Market Analysis and Prediction using Long Short Term Memory (LSTM)

In [2]:
# installing the yfinance modular
%pip install -q yfinance

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# installing the seaborn
%pip install seaborn

In [ ]:
# installing the pandas
%pip install pandas

In [ ]:
# installing the numpy
%pip install numpy